<li>Movie Name
<li>Release Date
<li>Rating
<li>Director,Screenplay
<li>Duration
<li>Genre
    
https://www.themoviedb.org/movie

Please see the **Function** section for the Commented Code

# One Movie

In [7]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

In [3]:
url = 'https://www.themoviedb.org/movie' #put your url in this quote

header = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36'}

source_data = requests.get(url, headers = header).text

In [4]:
soup_data = BeautifulSoup(source_data,'lxml')

In [5]:
movie1 = soup_data.find('div',class_='card style_1')

In [8]:
base_url = "https://www.themoviedb.org/movie/"
exp = str(movie1.find('a'))[30:38]  # get upto 7 characters, no of digits vary from 4 to 7
digits = re.findall("\d",exp) 
extension = "".join(digits)
movie_url = base_url + extension # movie url created

In [9]:
source_movie_data = requests.get(movie_url, headers = header).text

In [10]:
soup_movie_data = BeautifulSoup(source_movie_data,'lxml')

In [11]:
movie_name = soup_movie_data.find('h2').find('a').text
movie_name

'Wonka'

In [12]:
release_date = soup_movie_data.find('span',{'class':'release'}).text.replace('\n','').replace('(IN)','').strip()
print(release_date)

12/08/2023 (JP)


In [14]:
exp = '\d{2}.\d{2}.\d{4}'
ans3 = re.findall(exp,release_date)
print(ans3[0])

12/08/2023


In [16]:
rating = int(float(str(soup_movie_data.find('div',{'class':'user_score_chart'}))[69:73]))
rating

72

In [17]:
people = soup_movie_data.find('ol',{'class':'people no_image'}).find_all('li')
people

[<li class="profile">
 <p><a href="/person/101666-paul-king">Paul King</a></p>
 <p class="character">Director, Screenplay, Story</p>
 </li>,
 <li class="profile">
 <p><a href="/person/1299-roald-dahl">Roald Dahl</a></p>
 <p class="character">Characters</p>
 </li>,
 <li class="profile">
 <p><a href="/person/114253-simon-farnaby">Simon Farnaby</a></p>
 <p class="character">Screenplay</p>
 </li>]

In [18]:
for person in people:
    person_str = str(person)
    if 'Director' in person_str:
        director = person.find('a').text
print(director)

Paul King


In [19]:
runtime = soup_movie_data.find('span',class_='runtime').text.replace('\n','').strip()
runtime

'1h 57m'

In [20]:
genre = soup_movie_data.find('span',class_='genres').text.replace('\n','').replace('\xa0','')
genre

'Comedy,Family,Fantasy'

=====================================================================================================

# One Page

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

In [ ]:
# first url
url_main = 'https://www.themoviedb.org/movie'

# code for connecting to server without issues
header = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36'}

# get main url content
source_data = requests.get(url_main, headers = header).text

In [ ]:
soup_data = BeautifulSoup(source_data,'lxml')

# find all movies on the first page
movies = soup_data.find_all('div',class_='card style_1')

In [ ]:
# list will capture all movies
movies_list = []

# loop through each movie
for movie in movies:
    
    # get url of movie using href in html code
    # eg of a movie url https://www.themoviedb.org//movie/787699
    base_url = "https://www.themoviedb.org/movie/"
    exp = str(movie.find('a'))[30:38]  # get upto 7 characters, no of digits vary from 4 to 7
    digits = re.findall("\d",exp) 
    extension = "".join(digits)
    movie_url = base_url + extension # movie url created
    
    try:
        source_movie_data = requests.get(movie_url, headers = header).text
    except:
        source_movie_data = requests.get(movie_url, headers = header).text
    print(movie_url)
    soup_movie_data = BeautifulSoup(source_movie_data,'lxml')
    movie_name = soup_movie_data.find('h2').find('a').text
    release_date_1 = soup_movie_data.find('span',{'class':'release'}).text
    exp = '\d{2}.\d{2}.\d{4}'
    release_date = re.findall(exp,release_date_1)[0]
    #print(ans3)
    
    rating = float(str(soup_movie_data.find('div',{'class':'user_score_chart'}))[69:73])
    people = soup_movie_data.find('ol',{'class':'people no_image'}).find_all('li')
    for person in people:
        person_str = str(person)
        if 'Director' in person_str:
            director = person.find('a').text
    runtime = soup_movie_data.find('span',class_='runtime').text.replace('\n','').strip()
    genre = soup_movie_data.find('span',class_='genres').text.replace('\n','').replace('\xa0','')
    
    movie_dict = {
        'Movie Name':movie_name,
        'Release Date':release_date,
        'User Rating':rating,
        'Director':director,
        'Runtime':runtime,
        'Genres':genre
    }
    
    movies_list.append(movie_dict)
    #time.sleep(5)

In [ ]:
print(movies_list)

In [ ]:
df = pd.DataFrame(movies_list)
df.to_excel('Page1Moviesfinal.xlsx')

====================================================================================

# All Pages

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

In [2]:
# first url
url_main = 'https://www.themoviedb.org/movie'

# code for connecting to server without issues
header = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36'}

In [3]:
# get main url content
# create the list of URLs to loop through
url_list = [url_main]
url = 'https://www.themoviedb.org/movie'

# https://www.themoviedb.org/movie?page=2 ... https://www.themoviedb.org/movie?page=50
for i in range(2,51):
    temp_url = url + f'?page={i}'
    url_list.append(temp_url)

In [5]:
# list will capture all movies across all pages
final_movie_list = []

# for each url
for page in url_list:
    
    # get page code
    source_data = requests.get(page, headers = header).text
    soup_data = BeautifulSoup(source_data,'lxml')
    
    # find all movies on page
    movies = soup_data.find_all('div',class_='card style_1')
    
    # list will capture all movies on this one page
    movies_list = []
    
    # printing to keep check of status
    print(page) 
    print("======================================================================")
    
    # looping through each movie
    for movie in movies:
        
        # get url of movie using href in html code
        # eg of a movie url https://www.themoviedb.org/movie/787699
        base_url = "https://www.themoviedb.org/movie/"
        exp = str(movie.find('a'))[30:38]   # get upto 7 characters, no of digits vary from 4 to 7
        digits = re.findall("\d",exp)
        extension = "".join(digits)
        movie_url = base_url + extension    # movie url created
        
        # get content from the movie page using the newly created url
        source_movie_data = requests.get(movie_url, headers = header).text
        print(movie_url)  # printing to keep check of status
        soup_movie_data = BeautifulSoup(source_movie_data,'lxml')
        
        # 1. Movie Name
        movie_name = soup_movie_data.find('h2').find('a').text
        
        # 2. Release Date
        try:
            release_date_1 = soup_movie_data.find('span',{'class':'release'}).text
            # regular expression to get date
            exp = '\d{2}.\d{2}.\d{4}'
            release_date = re.findall(exp,release_date_1)[0]
        except:
            # Return none if release data not found
            release_date = None
            
        # 3. Rating
        try:
            rating = float(str(soup_movie_data.find('div',{'class':'user_score_chart'}))[69:73])
        except:
            # Return none if rating not found
            rating = None
        
        # 4. Director
        try:
            # make a list of all people and find Director if present
            people = soup_movie_data.find('ol',{'class':'people no_image'}).find_all('li')
            for person in people:
                person_str = str(person)
                if 'Director' in person_str:
                    director = person.find('a').text
        except:
            # Return none if Director not found
            director = None
        
        # 5. Runtime
        try:
            runtime = soup_movie_data.find('span',class_='runtime').text.replace('\n','').strip()
        except:
            # Return none if not found
            runtime = None
        
        # 6. Genres
        try:
            genre = soup_movie_data.find('span',class_='genres').text.replace('\n','').replace('\xa0','')
        except:
            # Return none if not found
            genre = None
        
        # capture the movie data in a dictionary    
        movie_dict = {
            'Movie Name':movie_name,
            'Release Date':release_date,
            'User Rating':rating,
            'Director':director,
            'Runtime':runtime,
            'Genres':genre
        }

        movies_list.append(movie_dict)
    final_movie_list.extend(movies_list)

https://www.themoviedb.org/movie
https://www.themoviedb.org/movie/787699
https://www.themoviedb.org/movie/466420
https://www.themoviedb.org/movie/901362
https://www.themoviedb.org/movie/897087
https://www.themoviedb.org/movie/572802
https://www.themoviedb.org/movie/1075794
https://www.themoviedb.org/movie/726209
https://www.themoviedb.org/movie/872585
https://www.themoviedb.org/movie/695721
https://www.themoviedb.org/movie/385687
https://www.themoviedb.org/movie/520758
https://www.themoviedb.org/movie/507089
https://www.themoviedb.org/movie/670292
https://www.themoviedb.org/movie/983507
https://www.themoviedb.org/movie/656156
https://www.themoviedb.org/movie/1146225
https://www.themoviedb.org/movie/1029575
https://www.themoviedb.org/movie/844416
https://www.themoviedb.org/movie/299054
https://www.themoviedb.org/movie/945937
https://www.themoviedb.org/movie?page=2
https://www.themoviedb.org/movie/566810
https://www.themoviedb.org/movie/844069
https://www.themoviedb.org/movie/346698
http

https://www.themoviedb.org/movie/40706
https://www.themoviedb.org/movie/760104
https://www.themoviedb.org/movie/930564
https://www.themoviedb.org/movie/744275
https://www.themoviedb.org/movie/414906
https://www.themoviedb.org/movie/1074080
https://www.themoviedb.org/movie/507086
https://www.themoviedb.org/movie/5255
https://www.themoviedb.org/movie/156022
https://www.themoviedb.org/movie/238
https://www.themoviedb.org/movie/1032948
https://www.themoviedb.org/movie/594767
https://www.themoviedb.org/movie/674
https://www.themoviedb.org/movie/508947
https://www.themoviedb.org/movie/597
https://www.themoviedb.org/movie/47971
https://www.themoviedb.org/movie/1076364
https://www.themoviedb.org/movie/19995
https://www.themoviedb.org/movie/977770
https://www.themoviedb.org/movie/1040148
https://www.themoviedb.org/movie?page=11
https://www.themoviedb.org/movie/1008102
https://www.themoviedb.org/movie/14160
https://www.themoviedb.org/movie/361743
https://www.themoviedb.org/movie/537915
https://w

https://www.themoviedb.org/movie/16237
https://www.themoviedb.org/movie/8966
https://www.themoviedb.org/movie/209112
https://www.themoviedb.org/movie/718930
https://www.themoviedb.org/movie/10138
https://www.themoviedb.org/movie/752623
https://www.themoviedb.org/movie/228150
https://www.themoviedb.org/movie/615457
https://www.themoviedb.org/movie/915935
https://www.themoviedb.org/movie/610150
https://www.themoviedb.org/movie/119450
https://www.themoviedb.org/movie/210577
https://www.themoviedb.org/movie/734253
https://www.themoviedb.org/movie/664767
https://www.themoviedb.org/movie/942881
https://www.themoviedb.org/movie/781732
https://www.themoviedb.org/movie/419548
https://www.themoviedb.org/movie/284053
https://www.themoviedb.org/movie?page=20
https://www.themoviedb.org/movie/536554
https://www.themoviedb.org/movie/1075519
https://www.themoviedb.org/movie/663712
https://www.themoviedb.org/movie/274479
https://www.themoviedb.org/movie/370172
https://www.themoviedb.org/movie/887870
ht

https://www.themoviedb.org/movie/823395
https://www.themoviedb.org/movie/315011
https://www.themoviedb.org/movie/190859
https://www.themoviedb.org/movie/1585
https://www.themoviedb.org/movie/862552
https://www.themoviedb.org/movie/20662
https://www.themoviedb.org/movie/1178734
https://www.themoviedb.org/movie/751237
https://www.themoviedb.org/movie/343144
https://www.themoviedb.org/movie/652
https://www.themoviedb.org/movie/4922
https://www.themoviedb.org/movie/207703
https://www.themoviedb.org/movie/28468
https://www.themoviedb.org/movie/820232
https://www.themoviedb.org/movie/300671
https://www.themoviedb.org/movie/1001884
https://www.themoviedb.org/movie?page=29
https://www.themoviedb.org/movie/832502
https://www.themoviedb.org/movie/383498
https://www.themoviedb.org/movie/474412
https://www.themoviedb.org/movie/877269
https://www.themoviedb.org/movie/1830
https://www.themoviedb.org/movie/44833
https://www.themoviedb.org/movie/573171
https://www.themoviedb.org/movie/628900
https://w

https://www.themoviedb.org/movie/10144
https://www.themoviedb.org/movie/1189198
https://www.themoviedb.org/movie/321612
https://www.themoviedb.org/movie/181533
https://www.themoviedb.org/movie/172385
https://www.themoviedb.org/movie/571265
https://www.themoviedb.org/movie/1039773
https://www.themoviedb.org/movie/551271
https://www.themoviedb.org/movie/853
https://www.themoviedb.org/movie/939338
https://www.themoviedb.org/movie/284052
https://www.themoviedb.org/movie/520763
https://www.themoviedb.org/movie/87
https://www.themoviedb.org/movie/61979
https://www.themoviedb.org/movie?page=38
https://www.themoviedb.org/movie/1422
https://www.themoviedb.org/movie/744
https://www.themoviedb.org/movie/10625
https://www.themoviedb.org/movie/637693
https://www.themoviedb.org/movie/351286
https://www.themoviedb.org/movie/166426
https://www.themoviedb.org/movie/981316
https://www.themoviedb.org/movie/379686
https://www.themoviedb.org/movie/436931
https://www.themoviedb.org/movie/714889
https://www.

https://www.themoviedb.org/movie/631842
https://www.themoviedb.org/movie/10991
https://www.themoviedb.org/movie/1062807
https://www.themoviedb.org/movie/396263
https://www.themoviedb.org/movie/877703
https://www.themoviedb.org/movie/1166714
https://www.themoviedb.org/movie/1250
https://www.themoviedb.org/movie/225886
https://www.themoviedb.org/movie/642885
https://www.themoviedb.org/movie/355338
https://www.themoviedb.org/movie/705861
https://www.themoviedb.org/movie?page=47
https://www.themoviedb.org/movie/294254
https://www.themoviedb.org/movie/546554
https://www.themoviedb.org/movie/428707
https://www.themoviedb.org/movie/291805
https://www.themoviedb.org/movie/27449
https://www.themoviedb.org/movie/98
https://www.themoviedb.org/movie/343611
https://www.themoviedb.org/movie/337401
https://www.themoviedb.org/movie/533514
https://www.themoviedb.org/movie/438650
https://www.themoviedb.org/movie/626872
https://www.themoviedb.org/movie/532408
https://www.themoviedb.org/movie/930094
https

In [6]:
# output the data in excel using a DataFrame
df = pd.DataFrame(final_movie_list)
df.to_excel('final_movie_list.xlsx')

===============================================================================================================

# Function

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

In [6]:
# first url
url_main = 'https://www.themoviedb.org/movie'

# code for connecting to server without issues
header = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36'}

# get main url content
# create the list of URLs to loop through
url_list = [url_main]
url = 'https://www.themoviedb.org/movie'

# https://www.themoviedb.org/movie?page=2 ... https://www.themoviedb.org/movie?page=50
for i in range(2,51):
    temp_url = url + f'?page={i}'
    url_list.append(temp_url)

In [7]:
#url_list

In [8]:
#url_list = url_list[:2] # to check for a limited number of urls/pages

In [9]:
def webscrapper_tmdb(url_list):
    '''
    Given a list of urls from TMDB the function finds the following for each movie on the URL:
    1. Movie Name
    2. Release Date
    3. Rating
    4. Director
    5. Runtime
    6. Genres
    
    Finally the function returns a dataframe containing the movie data
    '''

    # list will capture all movies across all pages
    final_movie_list = []

    # for each url
    for page in url_list:

        # get page code
        source_data = requests.get(page, headers = header).text
        soup_data = BeautifulSoup(source_data,'lxml')

        # find all movies on page
        movies = soup_data.find_all('div',class_='card style_1')

        # list will capture all movies on this one page
        movies_list = []

        # printing to keep check of status
        print(page) 
        print("======================================================================")

        # looping through each movie
        for movie in movies:

            # get url of movie using href in html code
            # eg of a movie url https://www.themoviedb.org/movie/787699
            base_url = "https://www.themoviedb.org/movie/"
            exp = str(movie.find('a'))[30:38]     # get upto 7 characters, no of digits vary from 4 to 7
            digits = re.findall("\d",exp)
            extension = "".join(digits)
            movie_url = base_url + extension      # movie url created

            # get content from the movie page using the newly created url
            source_movie_data = requests.get(movie_url, headers = header).text
            print(movie_url)                      # printing to keep check of status
            soup_movie_data = BeautifulSoup(source_movie_data,'lxml')

            # 1. Movie Name
            movie_name = soup_movie_data.find('h2').find('a').text

            # 2. Release Date
            try:
                release_date_1 = soup_movie_data.find('span',{'class':'release'}).text
                # regular expression to get date
                exp = '\d{2}.\d{2}.\d{4}'
                release_date = re.findall(exp,release_date_1)[0]
            except:
                # Return none if release data not found
                release_date = None

            # 3. Rating
            try:
                rating = float(str(soup_movie_data.find('div',{'class':'user_score_chart'}))[69:73])
            except:
                # Return none if rating not found
                rating = None

            # 4. Director
            try:
                # make a list of all people and find Director if present
                people = soup_movie_data.find('ol',{'class':'people no_image'}).find_all('li')
                # this list will capture all the directors of a movie (some movies have more than 1 director) 
                direct = []
                for person in people:
                    person_str = str(person)
                    if 'Director' in person_str:
                        director = person.find('a').text
                        direct.append(director)
            except:
                # Return none if Director not found
                direct = None
                        
            # 5. Runtime
            try:
                runtime = soup_movie_data.find('span',class_='runtime').text.replace('\n','').strip()
            except:
                # Return none if not found
                runtime = None

            # 6. Genres
            try:
                genre = soup_movie_data.find('span',class_='genres').text.replace('\n','').replace('\xa0','')
            except:
                # Return none if not found
                genre = None

            # capture the movie data in a dictionary    
            movie_dict = {
                'Movie Name':movie_name,
                'Release Date':release_date,
                'User Rating':rating,
                'Director':direct,
                'Runtime':runtime,
                'Genres':genre
            }

            movies_list.append(movie_dict)
        final_movie_list.extend(movies_list)
    
    # output the data in excel using a DataFrame
    df = pd.DataFrame(final_movie_list)
    df.to_excel('final_movies_list_updated.xlsx')
    return df

In [10]:
movie_df = webscrapper_tmdb(url_list)

https://www.themoviedb.org/movie
https://www.themoviedb.org/movie/848326
https://www.themoviedb.org/movie/695721
https://www.themoviedb.org/movie/1029575
https://www.themoviedb.org/movie/891699
https://www.themoviedb.org/movie/572802
https://www.themoviedb.org/movie/1071215
https://www.themoviedb.org/movie/507089
https://www.themoviedb.org/movie/897087
https://www.themoviedb.org/movie/670292
https://www.themoviedb.org/movie/872585
https://www.themoviedb.org/movie/299054
https://www.themoviedb.org/movie/787699
https://www.themoviedb.org/movie/1075794
https://www.themoviedb.org/movie/1189927
https://www.themoviedb.org/movie/901362
https://www.themoviedb.org/movie/520758
https://www.themoviedb.org/movie/1143183
https://www.themoviedb.org/movie/1131755
https://www.themoviedb.org/movie/853387
https://www.themoviedb.org/movie/502356
https://www.themoviedb.org/movie?page=2
https://www.themoviedb.org/movie/385687
https://www.themoviedb.org/movie/926393
https://www.themoviedb.org/movie/667538
h

https://www.themoviedb.org/movie?page=10
https://www.themoviedb.org/movie/566810
https://www.themoviedb.org/movie/532961
https://www.themoviedb.org/movie/1039690
https://www.themoviedb.org/movie/549945
https://www.themoviedb.org/movie/140300
https://www.themoviedb.org/movie/666277
https://www.themoviedb.org/movie/9502
https://www.themoviedb.org/movie/354912
https://www.themoviedb.org/movie/252
https://www.themoviedb.org/movie/150540
https://www.themoviedb.org/movie/12445
https://www.themoviedb.org/movie/790493
https://www.themoviedb.org/movie/457332
https://www.themoviedb.org/movie/493529
https://www.themoviedb.org/movie/287903
https://www.themoviedb.org/movie/1039960
https://www.themoviedb.org/movie/829557
https://www.themoviedb.org/movie/637649
https://www.themoviedb.org/movie/120
https://www.themoviedb.org/movie/299534
https://www.themoviedb.org/movie?page=11
https://www.themoviedb.org/movie/839369
https://www.themoviedb.org/movie/899112
https://www.themoviedb.org/movie/585511
https

https://www.themoviedb.org/movie/916224
https://www.themoviedb.org/movie/425
https://www.themoviedb.org/movie/1146143
https://www.themoviedb.org/movie/1159204
https://www.themoviedb.org/movie/554600
https://www.themoviedb.org/movie/1179316
https://www.themoviedb.org/movie/673593
https://www.themoviedb.org/movie/293660
https://www.themoviedb.org/movie/47570
https://www.themoviedb.org/movie/260514
https://www.themoviedb.org/movie/58
https://www.themoviedb.org/movie/383498
https://www.themoviedb.org/movie/716263
https://www.themoviedb.org/movie/1149381
https://www.themoviedb.org/movie/818647
https://www.themoviedb.org/movie/1049951
https://www.themoviedb.org/movie/10625
https://www.themoviedb.org/movie/13
https://www.themoviedb.org/movie/1041513
https://www.themoviedb.org/movie/324552
https://www.themoviedb.org/movie?page=20
https://www.themoviedb.org/movie/446893
https://www.themoviedb.org/movie/552688
https://www.themoviedb.org/movie/588228
https://www.themoviedb.org/movie/512200
https:

https://www.themoviedb.org/movie/680
https://www.themoviedb.org/movie/1064517
https://www.themoviedb.org/movie/652
https://www.themoviedb.org/movie/614934
https://www.themoviedb.org/movie/11036
https://www.themoviedb.org/movie/751237
https://www.themoviedb.org/movie/497698
https://www.themoviedb.org/movie/1930
https://www.themoviedb.org/movie/18239
https://www.themoviedb.org/movie/10144
https://www.themoviedb.org/movie/370172
https://www.themoviedb.org/movie/481848
https://www.themoviedb.org/movie/945729
https://www.themoviedb.org/movie/675445
https://www.themoviedb.org/movie/332662
https://www.themoviedb.org/movie/370663
https://www.themoviedb.org/movie/10193
https://www.themoviedb.org/movie/669525
https://www.themoviedb.org/movie/136799
https://www.themoviedb.org/movie?page=29
https://www.themoviedb.org/movie/545611
https://www.themoviedb.org/movie/12
https://www.themoviedb.org/movie/458156
https://www.themoviedb.org/movie/419680
https://www.themoviedb.org/movie/38365
https://www.the

https://www.themoviedb.org/movie/850165
https://www.themoviedb.org/movie/400160
https://www.themoviedb.org/movie/3933
https://www.themoviedb.org/movie/442249
https://www.themoviedb.org/movie/406990
https://www.themoviedb.org/movie/417859
https://www.themoviedb.org/movie/9714
https://www.themoviedb.org/movie/283366
https://www.themoviedb.org/movie/78
https://www.themoviedb.org/movie/464052
https://www.themoviedb.org/movie/420817
https://www.themoviedb.org/movie/7485
https://www.themoviedb.org/movie/284054
https://www.themoviedb.org/movie/68718
https://www.themoviedb.org/movie/727209
https://www.themoviedb.org/movie/262500
https://www.themoviedb.org/movie/862552
https://www.themoviedb.org/movie?page=38
https://www.themoviedb.org/movie/20662
https://www.themoviedb.org/movie/495764
https://www.themoviedb.org/movie/76338
https://www.themoviedb.org/movie/1158385
https://www.themoviedb.org/movie/1585
https://www.themoviedb.org/movie/964980
https://www.themoviedb.org/movie/303857
https://www.t

https://www.themoviedb.org/movie/453405
https://www.themoviedb.org/movie/562
https://www.themoviedb.org/movie/653346
https://www.themoviedb.org/movie/265712
https://www.themoviedb.org/movie/937278
https://www.themoviedb.org/movie/43347
https://www.themoviedb.org/movie/50268
https://www.themoviedb.org/movie/331482
https://www.themoviedb.org/movie/76163
https://www.themoviedb.org/movie/1115710
https://www.themoviedb.org/movie/57800
https://www.themoviedb.org/movie/273477
https://www.themoviedb.org/movie/805320
https://www.themoviedb.org/movie/1070830
https://www.themoviedb.org/movie/245891
https://www.themoviedb.org/movie?page=47
https://www.themoviedb.org/movie/450465
https://www.themoviedb.org/movie/873126
https://www.themoviedb.org/movie/812
https://www.themoviedb.org/movie/424
https://www.themoviedb.org/movie/297761
https://www.themoviedb.org/movie/796499
https://www.themoviedb.org/movie/910571
https://www.themoviedb.org/movie/49521
https://www.themoviedb.org/movie/893712
https://www

In [11]:
movie_df

,Movie Name,Release Date,User Rating,Director,Runtime,Genres
0,Rebel Moon - Part One: A Child of Fire,12/15/2023,65.0,[Zack Snyder],2h 14m,Science Fiction
1,The Hunger Games: The Ballad of Songbirds & Sn...,12/22/2023,73.0,[Francis Lawrence],2h 37m,"Drama,Science Fiction,Action"
2,The Family Plan,12/15/2023,74.0,[Simon Cellan Jones],1h 59m,"Action,Comedy"
3,Silent Night,12/01/2023,59.0,[John Woo],1h 44m,"Action,Crime"
4,Aquaman and the Lost Kingdom,01/12/2024,65.0,[James Wan],2h 4m,"Action,Adventure,Fantasy"
...,...,...,...,...,...,...
995,Ri¢hie Ri¢h,12/21/1994,59.0,[Donald Petrie],1h 35m,"Comedy,Family"
996,Sicario,04/09/2016,74.0,[Denis Villeneuve],2h 2m,"Action,Crime,Thriller"
997,Star Wars: The Rise of Skywalker,12/20/2019,64.0,[J.J. Abrams],2h 22m,"Adventure,Action,Science Fiction"
998,Léon: The Professional,03/25/1995,83.0,[Luc Besson],1h 51m,"Crime,Drama,Action"
